In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import math

B = 300
def bootstrap(x,y,B,fun,fun2):
  x = np.array(x)
  resampled_stat = []
  poly = []
  rbf = []
  for k in range(B):
    index = np.random.randint(0,len(x),len(x))
    index = np.sort(index)
    # sample = x[index]
    # print(k)
    sample = x[index]
    sample_y = y[index]

    bstat = fun(sample,sample_y,x)
    bstat2 = fun2(sample,sample_y,x)
    # print(np.shape(bstat))

    Err_poly = sample_y-bstat
    Err_rbf = sample_y-bstat2
    poly.append(Err_poly)
    rbf.append(Err_rbf)
    # print(np.shape(Err_poly))
    Err = np.mean(Err_poly**2 - Err_rbf**2)
    resampled_stat.append(Err)

  return resampled_stat,poly,rbf
def f_poly_func(x,y,x_original):
  lamb = 0.001
  d=20
  n=len(x)
  # x_original = np.reshape(x_original,(n,1))
  # II = np.ones(30)
  # II = np.reshape()
  x_train = np.dot(x,x.T)
  II = np.ones(n)
  II = np.reshape(II,(n,1))
  I = np.eye(n)
  ke_poly = (II+x_train)**d



  alpha_cap = np.dot((np.linalg.solve(ke_poly+lamb*I,I)),y)
  # print(alpha_cap)
  # x_original = np.reshape(x_original,(n,1))
  new_ke = lambda r: (1+np.dot(x,r))**d

  ke_poly = []
  for i in range(n):
    
    ke_poly.append(new_ke(x_original[i]).reshape(-1))
  # ke_poly = np.apply_along_axis(lambda r: (1+np.dot(x,r))**d,0,x_original)
  ke_poly = np.array(ke_poly)
  # print(np.shape(ke_poly))
  # ke_poly = ke_poly[:,:,0]
  
  f_cap_poly = np.dot(ke_poly,alpha_cap)
  # print(f_cap_poly)
 
  return f_cap_poly

def f_rbf_func(x,y,x_original):
  lamb = 0.0001
  gamma = 20
  n=len(x)
  I = np.eye(n)
  z = np.zeros((len(x),len(x)))
  for t in range(len(x)):
    z[t,:] = ((x[t,:] - x)).T    
  ke_rbf = np.exp(-gamma * (z**2))
  alpha_cap = np.dot((np.linalg.inv(ke_rbf+lamb*I)),y)

  krbf = lambda r: np.exp(- gamma * ((x - r)**2))

  ke_rbf = []
  for i in range(n):
    # x_train = x - x_original[i] ** 2
    ke_rbf.append(krbf(x_original[i]).reshape(-1))
  ke_rbf = np.array(ke_rbf)
  #ke_rbf = ke_rbf[:,:,0]


  f_cap_rbf = np.dot(ke_rbf,alpha_cap)
  return f_cap_rbf



########### Generating points ################
n = 1000
y = []
X = []

#x = np.random.uniform(size=n)
x = np.linspace(0,1,n)
eps = np.random.normal(0,1,n)
x = np.sort(x)
f = 4*np.sin(math.pi*x)*np.cos(6*math.pi*x*x)

y = f #+ eps

x = np.reshape(x,(n,1))


err,poly_err,rbf_err = bootstrap(x,y,B,f_poly_func,f_rbf_func)

# f_cap_poly_boot = bootstrap(x,y,B,f_poly_func)
# f_cap_rbf_boot = bootstrap(x,y,B,f_rbf_func)

# f_cap_poly_boot = np.reshape(f_cap_poly_boot,(B,n))
# f_cap_rbf_boot = np.reshape(f_cap_rbf_boot,(B,n))

# err = np.reshape(err,(B,n))


Uerr = np.quantile(err,0.95,axis=0)
Lerr = np.quantile(err,0.05,axis=0)

print("Upper percentile(95%):",Uerr)
print("Lower percentile(5%):",Lerr)
Max_poly = np.max(poly_err)
Max_rbf = np.max(rbf_err)

# if Max_poly > Max_rbf:
#   print("Poly has higher error")
# else:
#   print("RBF has higher error")

# Urbf = np.quantile(f_cap_rbf_boot,0.95,axis=0)
# Lrbf = np.quantile(f_cap_rbf_boot,0.05,axis=0)


# plt.plot(x,f,label='True')
# plt.plot(x,Uerr,label='Upper limit')
# plt.plot(x,Lerr,label='Lower limit')
# plt.xlabel("x")
# plt.ylabel("y")
# plt.legend()
# plt.title("")
# plt.show()

Upper percentile(95%): 1.1388371775195758
Lower percentile(5%): 0.36797165829892187
